In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
SEASONS = list(range(2022, 2026))

In [ ]:
SEASONS

In [4]:
DATA_DIRECTORY = "data"
STANDINGS_DIRECTORY = os.path.join(DATA_DIRECTORY, "standings")
SCORES_DIRECTORY = os.path.join(DATA_DIRECTORY, "scores")

In [5]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i) #increase time with each retry to account for delays/bans

        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch() #launch the browser and wait till its done launching to continue
                page = await browser.new_page() #essentially creates new tab in browser
                await page.goto(url, timeout = 10000) #send browser in that tab to a given page and wait till its finished
                print(await page.title()) #Know progress in scraping
                html = await page.inner_html(selector) # Select pieces of html we want
        except PlaywrightTimeout:
            print(f"Timeout error on this page: {url}")
            continue
        else:
            break
    return html # will return html unless three different retries fail, thus will return none

In [6]:
async def scrape_season(season): #Scrape box scores for a given season
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter") #CSS selector
    print(f"Fetched HTML for {season}: {html[:100] if html else 'None'}")

    if not html:
        print(f"HTML not retrieved for season {season}. Skipping")
        return
    soup = BeautifulSoup(html)          #Create Links for each of the standings pages
    links = soup.find_all("a")           
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]

    for url in standings_pages: #Save data and then process later
        save_path = os.path.join(STANDINGS_DIRECTORY, url.split("/")[-1]) # Given This: https://basketball-reference.com/leagues/NBA_2019_games-october.html , It will simply get this: NBA_2019_games-october.html
        if os.path.exists(save_path): # If already scraped, dont scrape again
            continue

        html = await get_html(url, "#all_schedule") #Get table that has boxscores
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
for season in SEASONS: #loop through seasons and wait till season has been scraped before moving on
    await scrape_season(season)

In [8]:
standings_files = os.listdir(STANDINGS_DIRECTORY)

In [9]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a") #Grab a tags from all the links
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores] 
    
    for url in box_scores:
        save_path = os.path.join(SCORES_DIRECTORY, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
    
        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [10]:
standings_files = [s for s in standings_files if ".html" in s]

In [ ]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIRECTORY, f)

    await scrape_game(filepath)